In [ ]:
from os import getcwd
from common.utils import * 

input_dir = getcwd() + '/img/input_samples/'
outp_dir = getcwd() + '/img/output_fft_noise_images/'

figsize = (10, 4)
figsize_wide = (12, 4)
figsize_narrow = (5, 5)
figsize_default = (6.4, 4.8)
cbar_loc = [0.9, 0.11, 0.02, 0.77]

%matplotlib inline

# Rubik's cube example

In [ ]:
cube = open_img(input_dir + 'cube_1.png', no_mean=False)
cube_fr = find_ft_2d(cube)

shape = cube.shape
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)

kernel = normalize(freq_sharp_round_filter_2d(xx, yy, 50, False))
eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=figsize)
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)

# Wrong scaling, but it gives a really interesting visual representation
ax2.imshow(find_ift_2d(cube_fr * kernel).real, cmap='gray', 
           vmin=0, vmax=np.max(np.log(np.absolute(cube_fr + eps))))
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=figsize)
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
ax2.imshow(np.log(np.absolute(cube_fr)), cmap='gray')
plt.show()

# Woman photo example

In [ ]:
woman = open_img(input_dir + 'woman.jpeg', no_mean=False)
woman_fr = find_ft_2d(woman)

shape = woman.shape
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)
x, y = freq_mesh_2d(shape)

fps = 30
seconds_num = 5

radius = 300
img_arr = [woman]
kernel_arr = []

for i in range(fps * seconds_num):
    kernel = freq_sharp_round_filter_2d(xx, yy, i / fps / seconds_num * radius, False)
    img_arr.append(find_ift_2d(woman_fr * kernel))

In [ ]:
# Animation performance is quite poor 
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
im = plt.imshow(img_arr[0], interpolation='none', aspect='equal', cmap='gray')


def animate_func(i): 
    im.set_array(img_arr[i])
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    # interval = 1000 / fps, # in ms
    # repeat=True
)

# anim.save(output_dir + 'women.gif', writer='pillow')

# Phase correlation example


In [ ]:
%matplotlib inline

f1 = open_img(input_dir + 'horse_1.png', no_mean=False)
f2 = open_img(input_dir + 'horse_translated_1.png', no_mean=False)

f1_freq = find_ft_2d(f1)
f2_freq = find_ft_2d(f2)

f1_magn = np.abs(f1_freq)
f2_magn = np.abs(f2_freq)

f1_phase = np.angle(f1_freq)
f2_phase = np.angle(f2_freq)

ncps = f1_freq * np.conj(f2_freq) / np.abs(f1_freq * f2_freq)
ncps_magn = np.abs(ncps)
img_shift = find_ift_2d(ncps)

f, axes = show_images(f1, f2)
plt.show()

In [ ]:
f, axes = show_images(np.log10(f1_magn), np.log10(f2_magn))
plt.show()

In [ ]:
f, axes = show_images(f1_phase, f2_phase)
plt.show()

In [ ]:
f, axes = show_images(img_shift, figsize=figsize_narrow)
plt.show()

In [ ]:
ind = np.unravel_index(np.argmax(img_shift, axis=None), img_shift.shape)
x_shift, y_shift = [i - j for i, j in zip(img_shift.shape, ind)]

print(f'X shift = {x_shift}')
print(f'Y shift = {y_shift}')

# Pink noise shifting in frequency space

In [ ]:
# Shift values
shape = (256, 256)
y_size, x_size = shape
yy, xx = freq_numbers_2d(shape)
x, y = freq_mesh_2d(shape)
dx, dy = 50, 25

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
noise_ft = find_ft_2d(whitenoise)
kernel = freq_pink_filter_2d(xx, yy, factor=2)

pink_noise_ft = noise_ft * kernel
pink_noise = find_ift_2d(pink_noise_ft)
pink_noise = normalize_img(pink_noise)
shift = np.exp(-1j * 2 * np.pi  * (x * dx / x_size + y * dy / y_size))

# Apply the phase shift along both axes
f_freq_shifted = pink_noise_ft * shift
shifted_pink_noise = find_ift_2d(f_freq_shifted)
shifted_pink_noise = normalize(shifted_pink_noise)

f, ax = show_images(pink_noise, shifted_pink_noise)
plt.show()

In [ ]:
%matplotlib notebook

# Shift values
dx, dy = 200, 0

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

# Generating cloud image
whitenoise = np.random.normal(0, 1, shape)
whitenoise_ft = find_ft_2d(whitenoise)
kernel = freq_pink_filter_2d(xx, yy, factor=2)
pink_ft = whitenoise_ft * kernel
pink_noise = normalize_img(find_ift_2d(pink_ft))
shift = np.exp(-1j * 2 * np.pi  * (x * dxx / x_size + y * dyy / y_size));

snapshots_freq = [pink_ft]
snapshots_spatial = [pink_noise]
for _ in range(fps * seconds_num):
    # print(np.sum(np.absolute(snapshots_freq[-1]) ** 2))
    snapshots_freq.append(snapshots_freq[-1] * shift)
    snapshots_spatial.append(find_ift_2d(snapshots_freq[-1]))

    
fig = plt.figure(figsize=(8,8))
frame = snapshots_spatial[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots_spatial[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    # interval = 1000 / fps, # in ms
)

# Example of XY smooth transition and shift

In [ ]:
%matplotlib inline

shape = (100, 100)
y_size, x_size = shape
yy, xx = freq_numbers_2d(shape)
depth = int(x_size * 0.3)
dx, dy = 50, 25

img = gen_cloud(x_size, y_size)
new_img = make_img_transition_xy(img, depth, outter_smooth=True)
shifted_img = shift_img_xy(new_img, dx, dy)

f, ax = show_images(img, new_img, shifted_img)
plt.show()

In [ ]:
%matplotlib notebook

# Shift values
dx, dy = 200, 0

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

dx_arr = [0]
dy_arr = [0]
for i in range(fps * seconds_num):
    dx_arr.append(dx_arr[-1] + dxx)
    dy_arr.append(dy_arr[-1] + dyy)
    

for i in range(fps * seconds_num):
    dx_arr[i] = round(dx_arr[i])
    dy_arr[i] = round(dy_arr[i])
    

x_mesh, y_mesh = np.meshgrid(xx, yy)
snapshots = []
for i in range(fps * seconds_num):
    snapshots.append(shift_img_xy(new_img, dx_arr[i], dy_arr[i]))

    
fig = plt.figure(figsize=(5,5))
frame = snapshots[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)